In [2]:
import boto3

In [3]:
import pandas as pd
from io import StringIO

/opt/conda/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/conda/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
forecast_list =  ['experiment2_14lags_20_12_2021_2d', 'experiment3_14lags_29_12_2021_2d', 
                  'experiment4_14lags_20_03_2023_2d', 'experiment3_14lags_05_05_2023_2d',
                  'experiment5_14lags_10_09_2023_2d', 'experiment6_14lags_25_11_2023_2d']

In [5]:
# Bucket name and common prefix
bucket_forecast = 'viamericas-datalake-dev-us-east-1-283731589572-analytics'
common_path = 'Forecast/Forecast_analysis/forecast_2d/'

In [6]:
# Inicializar el cliente de S3
client = boto3.client('s3')

In [7]:
# Initialize a list to store DataFrames
dfs = []

In [138]:
# Iterar sobre cada nombre en la lista forecast_list
for forecast_item in forecast_list:
    print(forecast_item)
    # Listar objetos en el bucket que comiencen con el nombre específico
    elements = client.list_objects(Bucket=bucket_forecast, Prefix=common_path)

    # Inicializar un DataFrame vacío para este grupo
    group_df = pd.DataFrame()

    # Iterar sobre cada objeto en la carpeta
    for obj in elements.get('Contents', []):
        # Verificar si el objeto es un archivo CSV
        if obj['Key'].endswith('.csv'):
            # Leer contenido CSV desde S3
            response = client.get_object(Bucket=bucket_forecast, Key=obj['Key'])
            csv_content = response['Body'].read().decode('utf-8')

            # Transformar contenido CSV a DataFrame
            df = pd.read_csv(StringIO(csv_content))
            
            # Agregar columna adicional para identificar el grupo
            df['group_id'] = forecast_item
            
    # Concatenar este DataFrame al DataFrame del grupo
    group_df = pd.concat([group_df, df], ignore_index=True)

experiment2_14lags_20_12_2021_2d
experiment3_14lags_29_12_2021_2d
experiment4_14lags_20_03_2023_2d
experiment3_14lags_05_05_2023_2d
experiment5_14lags_10_09_2023_2d
experiment6_14lags_25_11_2023_2d


In [139]:
# Unir todos los DataFrames en uno solo
final_df = pd.concat(dfs, ignore_index=True)

In [140]:
final_df[final_df['payer'] == 'elektra (mexico)'].sort_values('date')

,item_id,country,payer,date,p10,p50,p90,group_id,p5
6676,52,mexico,elektra (mexico),2022-11-26T00:00:00Z,6.676975e+06,7.868033e+06,9.174892e+06,experiment5_14lags_10_09_2023_2d,6.389297e+06
5308,52,mexico,elektra (mexico),2022-11-26T00:00:00Z,6.676975e+06,7.868033e+06,9.174892e+06,experiment3_14lags_05_05_2023_2d,6.389297e+06
8044,52,mexico,elektra (mexico),2022-11-26T00:00:00Z,6.676975e+06,7.868033e+06,9.174892e+06,experiment6_14lags_25_11_2023_2d,6.389297e+06
3940,52,mexico,elektra (mexico),2022-11-26T00:00:00Z,6.676975e+06,7.868033e+06,9.174892e+06,experiment4_14lags_20_03_2023_2d,6.389297e+06
2572,52,mexico,elektra (mexico),2022-11-26T00:00:00Z,6.676975e+06,7.868033e+06,9.174892e+06,experiment3_14lags_29_12_2021_2d,6.389297e+06
...,...,...,...,...,...,...,...,...,...
2405,57,mexico,elektra (mexico),2023-09-12T00:00:00Z,6.687194e+06,7.485700e+06,8.368893e+06,experiment3_14lags_29_12_2021_2d,5.579055e+06
6509,57,mexico,elektra (mexico),2023-09-12T00:00:00Z,6.687194e+06,7.485700e+06,8.368893e+06,experiment5_14lags_10_09_2023_2d,5.579055e+06
1037,57,mexico,elektra (mexico),2023-09-12T00:00:00Z,6.687194e+06,7.485700e+06,8.368893e+06,experiment2_14lags_20_12_2021_2d,5.579055e+06
7877,57,mexico,elektra (mexico),2023-09-12T00:00:00Z,6.687194e+06,7.485700e+06,8.368893e+06,experiment6_14lags_25_11_2023_2d,5.579055e+06


In [141]:
final_df['date'] = pd.to_datetime(final_df['date']).dt.date

In [143]:
final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

/tmp/ipykernel_20/2399063668.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  final_df = final_df.applymap(lambda x: x.upper() if isinstance(x, str) else x)


In [147]:
final_df.drop(['item_id', 'group_id'], axis=1, inplace=True)
#final_df.drop(['item_id'], axis=1, inplace=True)

In [148]:
final_df.head()

,country,payer,date,p10,p50,p90,p5
0,MEXICO,BANORTE (UT),2022-12-21,383671.181936,455333.540782,518021.416594,NaN
1,MEXICO,BANORTE (UT),2022-12-22,353858.016976,430334.002162,503642.791762,NaN
2,UNITED STATES,MASTERCARD ATM NETWORK,2022-12-21,1205.550605,2310.773896,3397.069665,NaN
3,UNITED STATES,MASTERCARD ATM NETWORK,2022-12-22,940.652864,1958.708337,3025.880813,NaN
4,BANGLADESH,MERCHANTRADE,2022-12-21,-83.643758,245.883363,565.531409,NaN


In [111]:
#final_df = final_df.iloc[:, 1:] # Recorto el ID

In [112]:
final_df['date'].unique()

array([datetime.date(2022, 12, 21), datetime.date(2022, 12, 22),
       datetime.date(2023, 5, 6), datetime.date(2023, 5, 7),
       datetime.date(2022, 12, 30), datetime.date(2022, 12, 31),
       datetime.date(2023, 3, 21), datetime.date(2023, 3, 22),
       datetime.date(2023, 9, 11), datetime.date(2023, 9, 12),
       datetime.date(2022, 11, 26), datetime.date(2022, 11, 27)],
      dtype=object)

In [16]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [219]:
#pip install awswrangler

In [21]:
import awswrangler as wr

In [22]:
# Read data from Athena table
df_daily_check = wr.athena.read_sql_query(
    f'SELECT * FROM {origin_name}.{database_name}.{table_name};',
    database=database_name
)

/opt/conda/lib/python3.10/site-packages/awswrangler/_distributed.py:104: FutureWarning: promote has been superseded by promote_options='default'.
  return cls.dispatch_func(func)(*args, **kw)


In [221]:
df_daily_check[df_daily_check['payer'] == 'BANCO AGRICOLA'].sort_values('date')

,payer,country,date,tx,amount,coupon_count
38551,BANCO AGRICOLA,EL SALVADOR,2020-12-31,633,145933.0000,550
13109,BANCO AGRICOLA,EL SALVADOR,2021-01-01,108,26346.7400,94
91675,BANCO AGRICOLA,EL SALVADOR,2021-01-02,544,164267.0000,481
18520,BANCO AGRICOLA,EL SALVADOR,2021-01-03,537,152693.9800,458
35173,BANCO AGRICOLA,EL SALVADOR,2021-01-04,703,246159.0000,591
...,...,...,...,...,...,...
80669,BANCO AGRICOLA,EL SALVADOR,2023-10-17,1062,370859.0000,11
77283,BANCO AGRICOLA,EL SALVADOR,2023-10-18,865,302292.3500,10
58216,BANCO AGRICOLA,EL SALVADOR,2023-10-19,962,348930.1400,16
34907,BANCO AGRICOLA,EL SALVADOR,2023-10-20,1607,568905.0000,23


In [149]:
# Unir los DataFrames en base a las columnas 'country', 'payer' y 'date'
df_forecast = df_daily_check.merge(final_df, on=['payer', 'country', 'date'], how='inner').drop_duplicates()

In [159]:
df_forecast['date'] = pd.to_datetime(df_forecast['date'],format='%Y-%m-%d')

In [151]:
date_list = ['2022-11-27','2022-12-22','2022-12-31','2023-03-22','2023-05-07','2023-09-12']

In [153]:
date_list = pd.to_datetime(date_list)

In [162]:
#FILTRO SOLAMENTE LOS DIAS N+2
df_forecast_dia2 = df_forecast[df_forecast['date'].isin(date_list)]

In [163]:
df_forecast_dia2[df_forecast_dia2['payer'] == 'BANCO AGRICOLA'].sort_values('date')

,payer,country,date,tx,amount,coupon_count,p10,p50,p90,p5
5742,BANCO AGRICOLA,EL SALVADOR,2022-11-27,952,305798.0000,7,281640.114636,331706.241051,379785.899338,269135.378922
2202,BANCO AGRICOLA,EL SALVADOR,2022-12-22,1877,564683.0000,29,320481.330069,376896.028039,430143.294573,NaN
2928,BANCO AGRICOLA,EL SALVADOR,2022-12-31,928,212987.0000,7,332877.104484,383332.195500,432320.475597,314902.202062
6228,BANCO AGRICOLA,EL SALVADOR,2023-03-22,857,339281.6600,10,275962.728386,314144.071818,364805.958876,263499.433622
1290,BANCO AGRICOLA,EL SALVADOR,2023-05-07,1645,555421.0000,20,395105.885987,481812.836452,568519.786918,370525.670341
4482,BANCO AGRICOLA,EL SALVADOR,2023-09-12,1096,383921.6000,8,347740.102465,405677.050364,457684.209201,280012.017901


In [177]:
# Grouping by 'payer' and 'country' concatenated for this level of granularity
df_forecast_dia2['payer_country'] = df_forecast_dia2['payer'] + '_' + df_forecast_dia2['country']

/tmp/ipykernel_20/740406771.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast_dia2['payer_country'] = df_forecast_dia2['payer'] + '_' + df_forecast_dia2['country']


In [178]:
df_forecast_dia2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 577 entries, 0 to 6966
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   payer          577 non-null    object        
 1   country        577 non-null    object        
 2   date           577 non-null    datetime64[ns]
 3   tx             577 non-null    Int64         
 4   amount         577 non-null    object        
 5   coupon_count   577 non-null    Int64         
 6   p10            577 non-null    float64       
 7   p50            577 non-null    float64       
 8   p90            577 non-null    float64       
 9   p5             481 non-null    float64       
 10  payer_country  577 non-null    object        
dtypes: Int64(2), datetime64[ns](1), float64(4), object(4)
memory usage: 55.2+ KB


In [179]:
##CALCULAMOS ERROR PORCENTUAL (MAPE) PARA CADA PAYER, ESTOO SERÁ INDICATIVO DEL WAPE PROMEDIO PARA TODOS LOS CORTES##
df_forecast_dia2['amount'] = pd.to_numeric(df_forecast_dia2['amount'])

/tmp/ipykernel_20/1413353924.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast_dia2['amount'] = pd.to_numeric(df_forecast_dia2['amount'])


In [180]:
#df_forecast['date'] = pd.to_datetime(df_forecast['date'])

In [181]:
df_forecast_dia2["pe"] = abs((df_forecast_dia2.amount - df_forecast_dia2['p50']) / df_forecast_dia2.amount)

/tmp/ipykernel_20/1238003223.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast_dia2["pe"] = abs((df_forecast_dia2.amount - df_forecast_dia2['p50']) / df_forecast_dia2.amount)


In [182]:
mape_payer_6_cuts=df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')

/tmp/ipykernel_20/395646793.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mape_payer_6_cuts=df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')


In [183]:
mape_payer_6_cuts.sort_values(by='mape', ascending=True).head(3)

,payer_country,mape
23,BANCO PICHINCHA (TN)_ECUADOR,0.007544
98,TELEDOLAR-NIC (TN)_NICARAGUA,0.047959
108,TRANSPAY_INDONESIA,0.086795


In [171]:
mape_payer_6_cuts.sort_values(by='mape', ascending=True).head(3)

,payer_country,mape
101,TELEDOLAR-NIC (TN)_NICARAGUA,0.035222
94,OXXO (TN)_MEXICO,0.096386
60,ENVIOS CON CLABE (APPRIZA)_MEXICO,0.096398


In [184]:
mape_country_6_cuts=df_forecast_dia2.groupby('country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')

/tmp/ipykernel_20/1309468246.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mape_country_6_cuts=df_forecast_dia2.groupby('country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')


In [185]:
mape_country_6_cuts.sort_values(by='mape', ascending=True).head(3)

,country,mape
28,NICARAGUA,0.193587
18,HAITI,0.207223
12,ECUADOR,0.224330


In [173]:
mape_country_6_cuts.sort_values(by='mape', ascending=True).head(3)

,country,mape
15,GERMANY,0.155671
29,NICARAGUA,0.172740
8,COLOMBIA,0.179992


In [186]:
##CALCULAMOS WAPE PARA CADA PAYER##
df_forecast_dia2["ae"] = abs((df_forecast_dia2.amount - df_forecast_dia2['p50']))
df_forecast_dia2["aep10"] = abs((df_forecast_dia2.amount - df_forecast_dia2['p10']))

/tmp/ipykernel_20/3766858438.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast_dia2["ae"] = abs((df_forecast_dia2.amount - df_forecast_dia2['p50']))


In [187]:
##WAPEs GENERALES CON 6 CORTES##
print(sum(df_forecast_dia2["ae"])/sum(df_forecast_dia2["amount"]))
print(sum(df_forecast_dia2["aep10"])/sum(df_forecast_dia2["amount"]))

0.1811141333584076


KeyError: 'aep10'

In [191]:
##WAPEs GENERALES CON 6 CORTES##
print(sum(df_forecast_dia2["ae"])/sum(df_forecast_dia2["amount"]))
print(sum(df_forecast_dia2["aep10"])/sum(df_forecast_dia2["amount"]))

0.1811141333584076
0.2150900084977965


In [81]:
##WAPEs GENERALES CON 6 CORTES##
print(sum(df_forecast_dia2["ae"])/sum(df_forecast_dia2["amount"]))
print(sum(df_forecast_dia2["aep10"])/sum(df_forecast_dia2["amount"]))

0.13890035552106245
0.19476317957326877


In [192]:
wape_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/271378938.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/271378938.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')


In [194]:
wape_payer_6_cuts.sort_values(by='wape', ascending=True).head(10)

,payer_country,wape
23,BANCO PICHINCHA (TN)_ECUADOR,0.007544
98,TELEDOLAR-NIC (TN)_NICARAGUA,0.047959
108,TRANSPAY_INDONESIA,0.086795
91,OXXO (TN)_MEXICO,0.123412
31,BANPAIS (RED CHAPINA)_HONDURAS,0.125134
57,ENVIOS CON CLABE (APPRIZA)_MEXICO,0.128135
39,CARIBE EXPRESS_DOMINICAN REPUBLIC,0.137154
111,VIAMERICAS USA DEPOSITS_UNITED STATES,0.139447
25,BANCO RENDIMENTO_BRAZIL,0.141891
30,BANORTE (UT)_MEXICO,0.144167


In [42]:
wape_payer_6_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
101,TELEDOLAR-NIC (TN)_NICARAGUA,0.032918
60,ENVIOS CON CLABE (APPRIZA)_MEXICO,0.096163
94,OXXO (TN)_MEXICO,0.097928


In [195]:
wape_p10_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/2740907148.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_p10_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/2740907148.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_p10_payer_6_cuts = df_forecast_dia2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')


In [196]:
wape_p10_payer_6_cuts.sort_values(by='wape', ascending=True).head(10)

,payer_country,wape
0,24XORO_MEXICO,0.036457
87,NAFA_MALI,0.050930
97,TELEDOLAR (TN)_COSTA RICA,0.089240
23,BANCO PICHINCHA (TN)_ECUADOR,0.109849
82,NAFA_BURKINA FASO,0.123809
57,ENVIOS CON CLABE (APPRIZA)_MEXICO,0.143738
35,BANSEFI - BANCO DEL BIENESTAR_MEXICO,0.149080
37,BBVA - BANCOMER (BTS)_MEXICO,0.153471
26,BANCOLOMBIA_COLOMBIA,0.158788
67,GRUPO EXITO (TN)_COLOMBIA,0.160584


In [44]:
wape_p10_payer_6_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
0,24XORO_MEXICO,0.091992
39,BBVA - BANCOMER (BTS)_MEXICO,0.122869
85,NAFA_BURKINA FASO,0.123809


In [197]:
wape_country_6_cuts = df_forecast_dia2.groupby('country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/817708580.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_country_6_cuts = df_forecast_dia2.groupby('country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')


In [198]:
wape_country_6_cuts.sort_values(by='wape', ascending=True).head(10)

,country,wape
38,UNITED STATES,0.143067
11,DOMINICAN REPUBLIC,0.143430
26,MEXICO,0.165545
4,BRAZIL,0.169045
16,GUATEMALA,0.181861
12,ECUADOR,0.190883
30,PERU,0.192993
28,NICARAGUA,0.194889
19,HONDURAS,0.201900
34,SIERRA LEONE,0.204654


In [46]:
wape_country_6_cuts.sort_values(by='wape', ascending=True).head(3)

,country,wape
27,MEXICO,0.125674
11,DOMINICAN REPUBLIC,0.129650
29,NICARAGUA,0.136759


In [199]:
wape_p10_country_6_cuts = df_forecast_dia2.groupby('country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/206627919.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_p10_country_6_cuts = df_forecast_dia2.groupby('country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')


In [200]:
wape_p10_country_6_cuts.sort_values(by='wape', ascending=True).head(10)

,country,wape
25,MALI,0.050930
8,COLOMBIA,0.168306
26,MEXICO,0.180883
9,COSTA RICA,0.202489
28,NICARAGUA,0.224507
11,DOMINICAN REPUBLIC,0.226342
12,ECUADOR,0.229326
16,GUATEMALA,0.240650
19,HONDURAS,0.241567
30,PERU,0.262739


In [48]:
wape_p10_country_6_cuts.sort_values(by='wape', ascending=True).head(3)

,country,wape
27,MEXICO,0.162479
8,COLOMBIA,0.196919
17,GUATEMALA,0.210405


In [49]:
###AHORA CALCULAMOS LOS MAPES Y WAPES PERO SIN CONTAR DOS CORTES FINES DE 2022 (RUIDOSOS)###

In [201]:
df_forecast2=df_forecast_dia2.copy()

In [202]:
df_forecast2=df_forecast2.loc[(df_forecast2.date!='2022-12-21') & (df_forecast2.date!='2022-12-22') & (df_forecast.date!='2022-12-31') & (df_forecast.date!='2022-12-30')]

In [203]:
df_forecast2.date.unique()

<DatetimeArray>
['2022-11-27 00:00:00', '2023-09-12 00:00:00', '2023-03-22 00:00:00',
 '2023-05-07 00:00:00']
Length: 4, dtype: datetime64[ns]

In [204]:
mape_payer_4_cuts=df_forecast2.groupby('payer_country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')

/tmp/ipykernel_20/1477661267.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mape_payer_4_cuts=df_forecast2.groupby('payer_country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')


In [205]:
mape_payer_4_cuts.sort_values(by='mape', ascending=True).head(3)

,payer_country,mape
23,BANCO PICHINCHA (TN)_ECUADOR,0.007544
98,TELEDOLAR-NIC (TN)_NICARAGUA,0.047959
91,OXXO (TN)_MEXICO,0.065865


In [54]:
mape_payer_4_cuts.sort_values(by='mape', ascending=True).head(3)

,payer_country,mape
101,TELEDOLAR-NIC (TN)_NICARAGUA,0.035222
94,OXXO (TN)_MEXICO,0.070342
42,CARIBE EXPRESS_DOMINICAN REPUBLIC,0.078237


In [206]:
wape_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/2779562617.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/2779562617.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')


In [207]:
wape_payer_4_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
23,BANCO PICHINCHA (TN)_ECUADOR,0.007544
39,CARIBE EXPRESS_DOMINICAN REPUBLIC,0.045743
98,TELEDOLAR-NIC (TN)_NICARAGUA,0.047959


In [56]:
wape_payer_4_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
101,TELEDOLAR-NIC (TN)_NICARAGUA,0.032918
42,CARIBE EXPRESS_DOMINICAN REPUBLIC,0.069413
94,OXXO (TN)_MEXICO,0.071102


In [208]:
wape_p10_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/1622549522.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_p10_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/1622549522.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_p10_payer_4_cuts = df_forecast2.groupby('payer_country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')


In [209]:
wape_p10_payer_4_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
35,BANSEFI - BANCO DEL BIENESTAR_MEXICO,0.023726
0,24XORO_MEXICO,0.036457
87,NAFA_MALI,0.050930


In [58]:
wape_p10_payer_4_cuts.sort_values(by='wape', ascending=True).head(3)

,payer_country,wape
37,BANSEFI - BANCO DEL BIENESTAR_MEXICO,0.023736
0,24XORO_MEXICO,0.091992
39,BBVA - BANCOMER (BTS)_MEXICO,0.113748


In [210]:
##WAPEs GENERALES CON 4 CORTES## - A DIA 2
print(sum(df_forecast2["ae"])/sum(df_forecast2["amount"]))
print(sum(df_forecast2["aep10"])/sum(df_forecast2["amount"]))

0.14376587441573332
0.18720581056774166


In [82]:
##WAPEs GENERALES CON 4 CORTES##
print(sum(df_forecast2["ae"])/sum(df_forecast2["amount"]))
print(sum(df_forecast2["aep10"])/sum(df_forecast2["amount"]))

0.12297167384510523
0.1858617361606469


In [211]:
mape_country_4_cuts=df_forecast2.groupby('country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')

/tmp/ipykernel_20/2276264139.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mape_country_4_cuts=df_forecast2.groupby('country').apply(lambda x: (x['pe'].mean())).reset_index(name='mape')


In [217]:
mape_country_4_cuts.sort_values(by='mape', ascending=True).head(10)

,country,mape
10,DOMINICAN REPUBLIC,0.096882
27,NICARAGUA,0.146013
11,ECUADOR,0.154874
17,HAITI,0.162612
29,PERU,0.167550
12,EL SALVADOR,0.181730
7,COLOMBIA,0.205509
25,MEXICO,0.261139
18,HONDURAS,0.265566
3,BRAZIL,0.380668


In [60]:
mape_country_4_cuts.sort_values(by='mape', ascending=True).head(3)

,country,mape
10,DOMINICAN REPUBLIC,0.125765
14,GERMANY,0.155671
7,COLOMBIA,0.155845


In [213]:
wape_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/488736708.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/488736708.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['ae'].sum() / x['amount'].sum())).reset_index(name='wape')


In [218]:
wape_country_4_cuts.sort_values(by='wape', ascending=True).head(10)

,country,wape
10,DOMINICAN REPUBLIC,0.054965
15,GUATEMALA,0.117619
37,UNITED STATES,0.132990
30,PHILIPPINES,0.137467
18,HONDURAS,0.140825
29,PERU,0.145913
11,ECUADOR,0.147994
12,EL SALVADOR,0.152231
25,MEXICO,0.152752
27,NICARAGUA,0.152884


In [62]:
wape_country_4_cuts.sort_values(by='wape', ascending=True).head(3)

,country,wape
10,DOMINICAN REPUBLIC,0.082936
16,GUATEMALA,0.105591
30,PERU,0.110789


In [215]:
wape_p10_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')

/tmp/ipykernel_20/976101415.py:1: RuntimeWarning: divide by zero encountered in scalar divide
  wape_p10_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')
/tmp/ipykernel_20/976101415.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  wape_p10_country_4_cuts = df_forecast2.groupby('country').apply(lambda x: (x['aep10'].sum() / x['amount'].sum())).reset_index(name='wape')


In [216]:
wape_p10_country_4_cuts.sort_values(by='wape', ascending=True).head(3)

,country,wape
24,MALI,0.050930
7,COLOMBIA,0.089440
8,COSTA RICA,0.134204


In [64]:
wape_p10_country_4_cuts.sort_values(by='wape', ascending=True).head(3)

,country,wape
26,MEXICO,0.165808
7,COLOMBIA,0.172130
16,GUATEMALA,0.189424


In [65]:
##EXPORTAMOS LOS 5 EXCEL##

In [66]:
df_forecast.to_excel('forecast_check.xlsx')

/tmp/ipykernel_20/3865528890.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  df_forecast.to_excel('forecast_check.xlsx')


In [67]:
wape_payer_6_cuts.sort_values(by='payer_country')
wape_payer_6_cuts.to_excel('wape_payers_6_cuts.xlsx')

/tmp/ipykernel_20/347351005.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_payer_6_cuts.to_excel('wape_payers_6_cuts.xlsx')


In [68]:
wape_p10_payer_6_cuts.sort_values(by='payer_country')
wape_p10_payer_6_cuts.to_excel('wape_p10_payers_6_cuts.xlsx')

/tmp/ipykernel_20/24501306.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_p10_payer_6_cuts.to_excel('wape_p10_payers_6_cuts.xlsx')


In [69]:
wape_country_6_cuts.sort_values(by='country')
wape_country_6_cuts.to_excel('wape_country_6_cuts.xlsx')

/tmp/ipykernel_20/2449405044.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_country_6_cuts.to_excel('wape_country_6_cuts.xlsx')


In [70]:
wape_p10_country_6_cuts.sort_values(by='country')
wape_p10_country_6_cuts.to_excel('wape_p10_country_6_cuts.xlsx')

/tmp/ipykernel_20/3343385022.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_p10_country_6_cuts.to_excel('wape_p10_country_6_cuts.xlsx')


In [71]:
wape_payer_4_cuts.sort_values(by='payer_country')
wape_payer_4_cuts.to_excel('wape_payers_4_cuts.xlsx')

/tmp/ipykernel_20/2445015265.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_payer_4_cuts.to_excel('wape_payers_4_cuts.xlsx')


In [72]:
wape_p10_payer_4_cuts.sort_values(by='payer_country')
wape_p10_payer_4_cuts.to_excel('wape_p10_payers_4_cuts.xlsx')

/tmp/ipykernel_20/2168826863.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_p10_payer_4_cuts.to_excel('wape_p10_payers_4_cuts.xlsx')


In [73]:
wape_country_4_cuts.sort_values(by='country')
wape_country_4_cuts.to_excel('wape_country_4_cuts.xlsx')

/tmp/ipykernel_20/4237733071.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_country_4_cuts.to_excel('wape_country_4_cuts.xlsx')


In [74]:
wape_p10_country_4_cuts.sort_values(by='country')
wape_p10_country_4_cuts.to_excel('wape_p10_country_4_cuts.xlsx')

/tmp/ipykernel_20/330825279.py:2: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  wape_p10_country_4_cuts.to_excel('wape_p10_country_4_cuts.xlsx')


In [75]:
mape_payer_6_cuts.to_excel('mape_payers_6_cuts.xlsx')

/tmp/ipykernel_20/924308559.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  mape_payer_6_cuts.to_excel('mape_payers_6_cuts.xlsx')


In [76]:
mape_country_6_cuts.to_excel('mape_country_6_cuts.xlsx')

/tmp/ipykernel_20/3382209971.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  mape_country_6_cuts.to_excel('mape_country_6_cuts.xlsx')


In [77]:
mape_payer_4_cuts.to_excel('mape_payers_4_cuts.xlsx')

/tmp/ipykernel_20/72755746.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  mape_payer_4_cuts.to_excel('mape_payers_4_cuts.xlsx')


In [78]:
mape_country_4_cuts.to_excel('mape_country_4_cuts.xlsx')

/tmp/ipykernel_20/233650276.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  mape_country_4_cuts.to_excel('mape_country_4_cuts.xlsx')
